In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import re
import json

# function for gathering all the links for every rental object on search result page

In [3]:
def object_details_links(links):
    links_str = str(links)
    links_str = links_str.replace('href="','')  
    hrefs = re.findall('\/angebot\/\d{6,9}', links_str)
    hrefs = set(hrefs)
    base_url_details = "https://www.immonet.de"
    full_url_details = [base_url_details + i for i in hrefs]
    return full_url_details

# loop for putting the links from every search result into one list

In [27]:
num = range(1,43)
full_list = []
for i in num:
    url = f"https://www.immonet.de/berlin/wohnung-mieten-seite{i}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    links = (soup.find_all('a', class_="flex-grow-1 display-flex"))
    x = object_details_links(links)
    full_list += x

# checking the length of the list for potential error and cleaning the list of potential duplicate links with set()

In [28]:
len(full_list)

1062

In [29]:
full_list = set(full_list)

In [30]:
len(full_list)

1062

# definining empty lists for the storing column information later used in pandas

In [31]:
descriptionL =[]
adressL=[]
zip_codeL=[]
price_coldL=[]
price_warmL=[]
depositL=[]
roomsL=[]
areaL=[]
yearbuildL=[]
objecturl=[]

# functions for gathering the data on rental object pages

In [33]:
def writedes():
    try:
        description = (soup.find('h1', id="expose-headline").text)
        description = description.replace('.',"")
        description = description.strip()
        descriptionL.append(description)
    except:
        descriptionL.append('NA')
     #   print(url)
     #   print('description error')

def writeadr():    
    try:
        adress = (soup.find('p', class_="text-100 pull-left").text)
        adress = adress.replace('\t','')
        adress = adress.replace('\n',' ')
        adress = adress.replace('\xa0','')
        adress = adress.replace('Auf Karte anzeigen','')
        adress = adress.strip() 
        adressL.append(adress)
    except:
        adressL.append('NA')
    #    print(url)
    #    print('adress error')
        
def writezip():
    try:
        adress = (soup.find('p', class_="text-100 pull-left").text)
        adress = adress.replace('\t','')
        adress = adress.replace('\n',' ')
        adress = adress.replace('\xa0','')
        adress = adress.replace('Auf Karte anzeigen','')
        adress = adress.strip() 
        zip_code = [int(s) for s in adress.split() if s.isdigit()]
        zip_code = str(zip_code)
        zip_codeL.append(zip_code)
    except:
        zip_codeL.append('NA')
      #  print(url)
      #  print('zip error')
        
def writepric():
        try:  
            price_cold = (soup.find('div', id="priceid_2").text)
            price_cold = price_cold.strip()
            price_cold = price_cold.replace('\xa0€','')
            price_coldL.append(price_cold)
        except:
            price_coldL.append('NA')
        #    print(url)
        #    print('price cold error')
            
def writeyea():
        try:  
            yearbuild = (soup.find('div', id='yearbuild').text)
            yearbuild = yearbuild.strip()
            yearbuildL.append(yearbuild)
        except:
            yearbuildL.append('NA')
         #   print(url)
         #   print('yearbuild error')

def writeare():
        try:  
            area = (soup.find('div', id='areaid_1').text)
            area = area.strip()
            area = area.replace('\xa0m²','')
            areaL.append(area)
        except:
            areaL.append('NA')
         #   print(url)
         #   print('area error')


def writeroo():
        try:  
            rooms = (soup.find('div', id='equipmentid_1').text)
            rooms = rooms.strip()
            roomsL.append(rooms)
        except:
            roomsL.append('NA')
           # print(url)
       #     print('rooms error')
            
def writepriw():
        try:  
            price_warm = (soup.find('div', id='priceid_4').text)
            price_warm = price_warm.strip()
            price_warm = price_warm.replace('\xa0€','')
            price_warmL.append(price_warm)
        except:
            price_warmL.append('NaN')
        #    print(url)
        #    print('price warm error')

# using a loop for the actual scraping, using predefined function to clear the data and store the data in predefined lists

In [34]:
for url in full_list:
    if requests.get(url).status_code==200:
        soup = BeautifulSoup(requests.get(url).text,'html.parser')
        objecturl.append(url)
        writedes()
        writeadr()
        writezip()
        writepric()
        writepriw()
        writeroo()
        writeare()
        writeyea()
        
    else:
        continue

# create an empty dataframe where the data will be stored later

In [ ]:
columns = ['description_short','adress','zip_code','price_cold','price_warm','rooms','area','yearbuild']
data = pd.DataFrame(columns=columns)

# executing a few steps for striping the zip code from the adress information 

In [84]:
zip_code = zip_codeL.copy()

In [85]:
zipc=[]
for i in zip_code:
    zipc.append(i[-6:-1])
        


# clearing the dataframe from any previous unsuccesful attemp of inputting data

In [94]:
data = data.iloc[0:0]

# transfer the data from the python lists into the dataframe

In [95]:

zipped = zip(descriptionL, adressL, zipc, price_coldL, price_warmL, roomsL, areaL, yearbuildL,objecturl)
for z in list(zipped):
    data = data.append({'description_short':z[0],'adress':z[1],'zip_code':z[2],'price_cold':z[3],'price_warm':z[4],'rooms':z[5],'area':z[6],'yearbuild':z[7],'url':z[8]},ignore_index=True)

# validation of the dataframe 

In [96]:
len(data)

1062

In [97]:
data

,description_short,adress,zip_code,price_cold,price_warm,rooms,area,yearbuild,url
0,2-Zimmer-Apartment in attraktivem Neubauprojek...,10785 Berlin,10785,1293.0,1499.5,2.0,58.77,2019,https://www.immonet.de/angebot/40783366
1,"Lofts, gerade neu modernisiert und bezugsfrei,...",15711 Königs Wusterhausen,15711,876.18,NaN,1.0,83.82,1996,https://www.immonet.de/angebot/41357831
2,Hübsche 3-Zi-Whg mit Balkon u EBK - Erstbezug ...,10435 Berlin,10435,1500.0,1700.0,2.0,72.0,1918,https://www.immonet.de/angebot/41668426
3,Traumwohnung mit Blick auf den Yachthafen in W...,Zum Großen Zernsee 6f 14542 Werder,14542,1135.0,NaN,3.0,98.6,2013,https://www.immonet.de/angebot/41393149
4,Luxus Studio-Apartment,Unter den Eichen 101 12203 Berlin,12203,1200.0,1360.0,2.0,80.05,2018,https://www.immonet.de/angebot/41362590
...,...,...,...,...,...,...,...,...,...
1057,Seniorenfreundliche Zweizimmerwohnung mit Seeb...,Klosterstrasse 9a 15344 Strausberg,15344,638.55,743.04,2.0,58.05,1900,https://www.immonet.de/angebot/41611787
1058,schöne neu sanierte 2 Zimmer Wohnung in Friedr...,12587 Berlin,12587,606.0,NaN,2.0,62.38,1906,https://www.immonet.de/angebot/41641060
1059,"3 Zimmer, frisch saniert, mit Balkon",Otto-Heese-Straße 17 14641 Nauen,14641,420.0,560.0,3.0,57.42,1978,https://www.immonet.de/angebot/41482964
1060,"Vollmöblierte Komfort-Apartments im ""Urban Liv...",Balatonstr. 1/1a 10319 Berlin,10319,480.0,NaN,1.0,23.5,2018,https://www.immonet.de/angebot/41363118


# export the dataframe to a csv file

In [98]:
data.to_csv('data_with_zip.csv')